# COVID-19 Interactive Analysis Dashboard

In [26]:
# importing libraries

# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, SelectMultiple
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
# import folium
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import seaborn as sns
import ipywidgets as widgets

In [27]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
us_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
us_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [28]:
# confirmed_df.head()

In [29]:
# data cleaning

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
us_confirmed_df.columns = map(str.lower, us_confirmed_df.columns)
us_death_df.columns = map(str.lower, us_death_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
us_confirmed_df = us_confirmed_df.rename(columns={'province_state': 'state', 'admin2': 'county', 'country_region': 'country'})
us_death_df = us_death_df.rename(columns={'province_state': 'state', 'admin2': 'county', 'country_region': 'country'})
# drop rows with nulls
us_confirmed_df = us_confirmed_df.dropna(axis = 0, how ='any') 
us_death_df = us_death_df.dropna(axis = 0, how ='any') 

In [30]:
# death_df.head()

In [31]:
# death_daily_df = death_df.copy(deep=True)
# for col in range(6, len(death_daily_df.columns)):
#     death_daily_df[death_daily_df.columns[col]] = death_df[death_df.columns[col]] - death_df[death_df.columns[col-1]]

In [32]:
# death_daily_df.head()

In [33]:
# compute daily cases
confirmed_daily_df = confirmed_df.copy(deep=True)
# for row in range(len(confirmed_daily_df)):
for col in range(6, len(confirmed_daily_df.columns)):
    confirmed_daily_df[confirmed_daily_df.columns[col]] = confirmed_df[confirmed_df.columns[col]] - confirmed_df[confirmed_df.columns[col-1]]
#         confirmed_daily_df.iloc[row, col] = confirmed_df.iloc[row, col] - confirmed_df.iloc[row, col-1]

death_daily_df = death_df.copy(deep=True)
# for row in range(len(death_daily_df)):
for col in range(6, len(death_daily_df.columns)):
    death_daily_df[death_daily_df.columns[col]] = death_df[death_df.columns[col]] - death_df[death_df.columns[col-1]]
#         death_daily_df.iloc[row, col] = death_df.iloc[row, col] - death_df.iloc[row, col-1]
#         death_daily_df[col] = death_df[col] - death_df[col-1]

us_confirmed_daily_df = us_confirmed_df.copy(deep=True)
# for row in range(len(us_confirmed_daily_df)):
for col in range(12, len(us_confirmed_daily_df.columns)):
    us_confirmed_daily_df[us_confirmed_daily_df.columns[col]] = us_confirmed_df[us_confirmed_df.columns[col]] - us_confirmed_df[us_confirmed_df.columns[col-1]]
#         us_confirmed_daily_df.iloc[row, col] = us_confirmed_df.iloc[row, col] - us_confirmed_df.iloc[row, col-1]

us_death_daily_df = us_death_df.copy(deep=True)
# for row in range(len(us_death_daily_df)):
for col in range(12, len(us_death_daily_df.columns)):
    us_death_daily_df[us_death_daily_df.columns[col]] = us_death_df[us_death_df.columns[col]] - us_death_df[us_death_df.columns[col-1]]
#         us_death_daily_df.iloc[row, col] = us_death_df.iloc[row, col] - us_death_df.iloc[row, col-1]       

In [34]:
#us_confirmed_df.groupby('state').sum()

In [35]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [36]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

# COVID-19 Confirmed/Death/Recovered cases by countries
## Enter number of countries you want the data for

In [37]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

#interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [38]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

# Slide to check for the worst hit countries

In [39]:
# plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 800
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Check the details of your country or the World

In [40]:
#confirmed_df.head()

In [41]:
def plot_cases_of_a_country(x):
    labels = ['confirmed', 'deaths', 'confirmed', 'deaths']
    colors = ['blue', 'red', 'blue', 'red']
    mode_size = [6, 6, 6, 6]
    line_size = [3, 3, 3, 3]

    chartType = ['bar', 'bar', 'line', 'line']
    df_list = [confirmed_daily_df, death_daily_df, confirmed_df, death_df]

#     df_list = [confirmed_df, death_df]
#     df_list = [confirmed_df, death_df, confirmed_daily_df, death_daily_df]
    
    #fig = go.Figure();
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    yMaxSecondary = max(np.sum(np.asarray(confirmed_daily_df[confirmed_daily_df['country'] == x[0]].iloc[:,60:]),axis = 0)) * 2.0
#     yMaxSecondary = confirmed_daily_df['country'] == x.max().max() * 2
    fig.update_yaxes(range=[3, yMaxSecondary], secondary_y=True, showgrid=False)
    
    for i, df in enumerate(df_list):
        for country in x:
#             print(str(df[df['country'] == country].max().max()))
            if country == 'World' or country == 'world':
                x_data = np.array(list(df.iloc[:, 60:].columns))
                y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)

            else:    
                x_data = np.array(list(df.iloc[:, 60:].columns))
                y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,60:]),axis = 0)
                
            if (chartType[i] == 'line'):
                fig.add_trace(go.Scatter(x = x_data, y = y_data, line_shape = 'linear', 
                                         name=country + ' ' + labels[i] + ' total (' + str(max(y_data)) + ')'), secondary_y = False)
                #line_shape - linear/spline
            elif (chartType[i] == 'bar'):
                fig.add_trace(go.Bar(x = x_data, y = y_data, name=country + ' ' + labels[i] + ' daily'), secondary_y = True)
    #             fig = go.Figure(data=[go.Bar(name='xxx', x=x_data, y=y_data)])
    
    fig.update_layout(
        title="COVID 19 cases by Country ",
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

country_list = confirmed_df.country.unique().tolist()
country_list.sort()
startValue = ['US',]

countrySelection = SelectMultiple(
    options = country_list, # What will be available in wellSelection
    value = startValue, # What will first be chosen.  Here we are chossing all
    rows=10, # How many rows will be displayed initially
    description='Countries', # Text outside of the widget
    disabled=False # If 'True', the widget will not be usable
)

interact(plot_cases_of_a_country, x=countrySelection)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(SelectMultiple(description='Countries', index=(174,), options=('Afghanistan', 'Albania',…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Details by State

In [42]:
def plot_cases_by_state(x):
    labels = ['deaths', 'confirmed', 'deaths', 'confirmed']
    colors = ['blue', 'red', 'blue', 'red']
    mode_size = [6, 6, 6, 6]
    line_size = [3, 3, 3, 3]

    chartType = ['bar', 'bar', 'line', 'line']

#     df_list = [us_death_daily_df, us_confirmed_daily_df]
    df_list = [us_death_daily_df, us_confirmed_daily_df, us_death_df, us_confirmed_df]
    
    #fig = go.Figure();
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    print()
    yMaxSecondary = max(np.sum(np.asarray(us_confirmed_daily_df[us_confirmed_daily_df['state'] == x[0]].iloc[:,60:]),axis = 0)) * 2.0
    fig.update_yaxes(range=[3, yMaxSecondary], secondary_y=True, showgrid=False)
    
    for i, df in enumerate(df_list):
        for state in x:
            x_data = np.array(list(df.iloc[:, 60:].columns))
            y_data = np.sum(np.asarray(df[df['state'] == state].iloc[:,60:]),axis = 0)

            if (chartType[i] == 'line'):
                fig.add_trace(go.Scatter(x = x_data, y = y_data, line_shape = 'linear', 
                                         name=state + ' ' + labels[i] + ' total (' + str(max(y_data)) + ')'), secondary_y = False)
                #line_shape - linear/spline
            elif (chartType[i] == 'bar'):
                fig.add_trace(go.Bar(x = x_data, y = y_data, name=state + ' ' + labels[i] + ' daily'), secondary_y = True)
    #             fig = go.Figure(data=[go.Bar(name='xxx', x=x_data, y=y_data)])
    
    fig.update_layout(
        title="COVID 19 cases by State",
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

state_list = us_confirmed_df.state.unique().tolist()
state_list.sort()
startValue = ['Texas',]

stateSelection = SelectMultiple(
    options = state_list, # What will be available in wellSelection
    value = startValue, # What will first be chosen.  Here we are chossing all
    rows=10, # How many rows will be displayed initially
    description='States', # Text outside of the widget
    disabled=False # If 'True', the widget will not be usable
)

#states_list = us_confirmed_df.state.unique()
interact(plot_cases_by_state, x=stateSelection)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(SelectMultiple(description='States', index=(43,), options=('Alabama', 'Alaska', 'Arizona…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [43]:
us_death_df.drop(us_death_df[us_death_df['county'].str.contains("Out of")].index, inplace = True) 
#us_confirmed_df.drop(us_confirmed_df[us_confirmed_df['county'].str.contains("Out of")].index, inplace = True) 

# Details by County

In [44]:
def plot_cases_by_county(x):
    labels = ['deaths', 'confirmed', 'deaths', 'confirmed']
    colors = ['blue', 'red', 'blue', 'red']
    mode_size = [6, 6, 6, 6]
    line_size = [3, 3, 3, 3]

    chartType = ['bar', 'bar', 'line', 'line']

#     df_list = [us_death_daily_df, us_confirmed_daily_df]
    df_list = [us_death_daily_df, us_confirmed_daily_df, us_death_df, us_confirmed_df]
    
    #fig = go.Figure();
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    print()
    yMaxSecondary = max(np.sum(np.asarray(us_confirmed_daily_df[us_confirmed_daily_df['county'] == x[0]].iloc[:,60:]),axis = 0)) * 2.0
    fig.update_yaxes(range=[3, yMaxSecondary], secondary_y=True, showgrid=False)
    
    for i, df in enumerate(df_list):
        for county in x:
            x_data = np.array(list(df.iloc[:, 60:].columns))
            y_data = np.sum(np.asarray(df[df['county'] == county].iloc[:,60:]),axis = 0)

            if (chartType[i] == 'line'):
                fig.add_trace(go.Scatter(x = x_data, y = y_data, line_shape = 'linear', 
                                         name=county + ' ' + labels[i] + ' total (' + str(max(y_data)) + ')'), secondary_y = False)
                #line_shape - linear/spline
            elif (chartType[i] == 'bar'):
                fig.add_trace(go.Bar(x = x_data, y = y_data, name=county + ' ' + labels[i] + ' daily'), secondary_y = True)
    #             fig = go.Figure(data=[go.Bar(name='xxx', x=x_data, y=y_data)])
    
    fig.update_layout(
        title="COVID 19 cases by County",
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();
    
county_list = us_confirmed_df.county.unique().tolist()
#print(county_list)
county_list.sort()
startValue = ['Harris',]

countySelection = SelectMultiple(
    options = county_list, # What will be available in wellSelection
    value = startValue, # What will first be chosen.  Here we are chossing all
    rows=10, # How many rows will be displayed initially
    description='Counties', # Text outside of the widget
    disabled=False # If 'True', the widget will not be usable
)

interact(plot_cases_by_county, x=countySelection)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(SelectMultiple(description='Counties', index=(725,), options=('Abbeville', 'Acadia', 'Ac…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# 10 worst hit countries - Confirmed cases

In [45]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Top 10 worst affected countries - Confirmed Cases", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

# 10 worst hit countries - Death cases

In [46]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Top 10 worst affected countries - Deaths", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

# Worst hit countries - Recovering cases

In [47]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "recovered",
    title= "Top 10 worst affected countries - Recovering", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)